# Objective:
- Understand when and why employees are most likely to leave (people analytics)
- Build a model to predict employee retention
- Create a table with date, employee_headcount (# employed on that specific day), company_id
- Derive main driving factors for employee churn
- Consider potential additional variables that can aid in predicting employee retention

In [21]:

import  warnings
warnings.simplefilter('ignore')
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import graphviz
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder

%matplotlib inline

In [4]:

data = pd.read_csv('data/employee_retention_data.csv', parse_dates=['join_date', 'quit_date'])
data.head()

,employee_id,company_id,dept,seniority,salary,join_date,quit_date
0,13021.0,7,customer_service,28,89000.0,2014-03-24,2015-10-30
1,825355.0,7,marketing,20,183000.0,2013-04-29,2014-04-04
2,927315.0,4,marketing,14,101000.0,2014-10-13,NaT
3,662910.0,7,customer_service,20,115000.0,2012-05-14,2013-06-07
4,256971.0,2,data_science,23,276000.0,2011-10-17,2014-08-22


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24702 entries, 0 to 24701
Data columns (total 7 columns):
employee_id    24702 non-null float64
company_id     24702 non-null int64
dept           24702 non-null object
seniority      24702 non-null int64
salary         24702 non-null float64
join_date      24702 non-null datetime64[ns]
quit_date      13510 non-null datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int64(2), object(1)
memory usage: 1.3+ MB


In [7]:
data.describe(include = 'all')

,employee_id,company_id,dept,seniority,salary,join_date,quit_date
count,24702.000000,24702.000000,24702,24702.000000,24702.000000,24702,13510
unique,NaN,NaN,6,NaN,NaN,995,664
top,NaN,NaN,customer_service,NaN,NaN,2012-01-03 00:00:00,2015-05-08 00:00:00
freq,NaN,NaN,9180,NaN,NaN,105,111
first,NaN,NaN,NaN,NaN,NaN,2011-01-24 00:00:00,2011-10-13 00:00:00
last,NaN,NaN,NaN,NaN,NaN,2015-12-10 00:00:00,2015-12-09 00:00:00
mean,501604.403530,3.426969,NaN,14.127803,138183.345478,NaN,NaN
std,288909.026101,2.700011,NaN,8.089520,76058.184573,NaN,NaN
min,36.000000,1.000000,NaN,1.000000,17000.000000,NaN,NaN
25%,250133.750000,1.000000,NaN,7.000000,79000.000000,NaN,NaN


In [10]:
# Make sure employee ids are unique
assert data['employee_id'].nunique() == len(data)

In [11]:

# Null information
data.isnull().sum()

employee_id        0
company_id         0
dept               0
seniority          0
salary             0
join_date          0
quit_date      11192
dtype: int64

## Create Table for day, employee_headcount, and company_id

In [12]:
#check seniority outlier
data.sort_values('seniority', ascending=False).head()

,employee_id,company_id,dept,seniority,salary,join_date,quit_date
24701,604052.0,1,marketing,99,185000.0,2011-07-26,2013-12-06
24700,97289.0,10,engineer,98,266000.0,2011-12-13,2015-01-09
11717,426218.0,2,customer_service,29,118000.0,2011-03-02,2013-07-12
12013,889253.0,2,engineer,29,318000.0,2011-09-12,NaT
647,664204.0,8,marketing,29,147000.0,2014-02-03,2015-02-20


The top two items are the outliners

In [13]:
data.sort_values('salary', ascending=False).head(10)

,employee_id,company_id,dept,seniority,salary,join_date,quit_date
10689,62540.0,2,engineer,23,408000.0,2013-08-26,NaT
7919,517439.0,2,engineer,25,406000.0,2013-09-16,NaT
3172,965750.0,1,data_science,19,398000.0,2015-05-04,NaT
4150,707585.0,1,engineer,22,393000.0,2013-09-16,NaT
6638,460908.0,1,engineer,24,386000.0,2014-11-03,NaT
8823,820823.0,1,data_science,23,381000.0,2014-11-03,NaT
4432,324366.0,2,data_science,26,381000.0,2014-12-17,NaT
12496,921637.0,1,engineer,16,379000.0,2012-09-17,2015-09-25
20191,962125.0,1,data_science,26,378000.0,2011-12-19,2014-08-29
7487,807316.0,1,data_science,20,377000.0,2015-10-26,NaT


In [14]:
#covert type to int
data = data.astype({'employee_id':'int64'})

In [15]:
data.set_index('employee_id', inplace=True)

In [17]:
data.head()

,company_id,dept,seniority,salary,join_date,quit_date
employee_id,,,,,,
13021,7,customer_service,28,89000.0,2014-03-24,2015-10-30
825355,7,marketing,20,183000.0,2013-04-29,2014-04-04
927315,4,marketing,14,101000.0,2014-10-13,NaT
662910,7,customer_service,20,115000.0,2012-05-14,2013-06-07
256971,2,data_science,23,276000.0,2011-10-17,2014-08-22


In [19]:
# Create an intermediate table with the extreme seniority values removed
data = data.drop([604052, 97289])

In [22]:
# Define the start date and end dates
start_date = datetime.datetime.strptime('2011-01-24', '%Y-%m-%d')
end_date = datetime.datetime.strptime('2015-12-13', '%Y-%m-%d')

In [29]:
assert data['join_date'].max()<=end_date
assert data['join_date'].min()>=start_date
assert data['quit_date'].max()<=end_date
assert data['quit_date'].min()>start_date

In [24]:
1+1

2

In [ ]:
unique_date = pd.date_range(start=start_date, end = end_date)
unique_columns = sorted(data['company_id'].unique(0))

day = []
company = []
headcount = []

#loop through date and company id

for one_day in unique_date:
    for one_cp in unique_company:
         

In [32]:
data[['company_id','dept']]

,company_id,dept
employee_id,,
13021,7,customer_service
825355,7,marketing
927315,4,marketing
662910,7,customer_service
256971,2,data_science
...,...,...
63273,1,data_science
94644,6,marketing
549847,3,customer_service
